# Limpieza de los datos
Es crítico asegurarse de examinar y preprocesar los datasets antes de utilizarlos en un modelo de Machine Learning (ML). En este notebook explicaré cómo hacer las siguientes tareas:

* Remover e imputar valores perdidos en los datasets
* Transformar datos categóricos a una forma (shape) que pueda entender los algoritmos de ML
* Seleccionar las características relevantes de los datos para construir el modelo

## Tratando con datos perdidos
Los datos perdidos son consecuencia del proceso de recolección de los datos. Normalmente se tratan de campos en blaco o con valores NaN (Not A Number).

A continuación crearemos un dataset (demo) para ver como tratar con los datos perdidos. Simularemos que los datos provienen de un archivo csv usando la función *StringIO* de la librería *io* 

In [1]:
import pandas as pd
from io import StringIO
csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
0.0,11.0,12.0,'''
df = pd.read_csv(StringIO(csv_data))

In [2]:
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,0.0,11.0,12.0,NaN


In [3]:
# Usamos la función isnull de los DataFrame de Pandas para ver una foto del DataFrame en término
# de los valores nulos que están presentes. En cada campo donde aparece True, se indica que ese valor
# es nulo.
df.isnull()

,A,B,C,D
0,False,False,False,False
1,False,False,True,False
2,False,False,False,True


In [4]:
# Usamos la función sum para contar el número de nulos por cada columna del DataFrame
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

### Eliminando muestras (registros o filas) o características (columnas) con valores perdidos
Una de las maneras de tratar los valores perdidos es eliminandolos del dataset, bien sea por filas o por columnas.

In [8]:
# Eliminar las filas que tienen al menos un nulo
df.dropna()

,A,B,C,D
0,1.0,2.0,3.0,4.0


**Nota**: df.dropna() es equivalente a df.dropna(axis = 0)

In [6]:
# Eliminar las columnas que tienen al menos un nulo
df.dropna(axis = 1)

,A,B
0,1.0,2.0
1,5.0,6.0
2,0.0,11.0


**Nota**: El método dropna soporta varios parámetros adicionales:

In [9]:
# Solo elimina las filas donde todas las columnas son NaN
df.dropna(how = 'all')

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,0.0,11.0,12.0,NaN


In [10]:
# Elimina las filas que NO tienen al menos 4 valores que NO sean NaN
df.dropna(thresh = 4)

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [13]:
# Solo elimina las filas donde los NaN aparecen en una lista de columnas específicas
df.dropna(subset=['C'])

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,0.0,11.0,12.0,NaN


### Imputando valores perdidos
La eliminación de valores perdidos no siempre es una buena opción. En este caso debemos hablar de imputación de valores perdidos.

La imputación se refiere a la aplicación de técnicas de interpolación para estimar el valor de un campo a partir de otros valores presentes en el dataset. Una de las técnicas más utilizadas es la **imputación por la media**, en la que se reemplaza el valor perdido por el valor de la media de la columna completa.

Una manera conveniente de hacer *imputación* en los datos es usando la librería **scikit-learn** de Python.

In [30]:
from sklearn.impute import SimpleImputer
import numpy as np
imr = SimpleImputer(missing_values=np.nan, strategy='mean')
# El método fit se utiliza para aprender los parámetros desde el conjunto de datos de entrenamiento.
imr = imr.fit(df)
# El método transform utiliza los parámetros aprendidos por el método fit para aplicarlo a los datos
imputed_data = imr.transform(df.values)

In [31]:
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [ 0. , 11. , 12. ,  6. ]])

**Nota**: El parámetro *strategy* tiene varias opciones, las cuales se describen a continuación:
* Si el valor es “mean”, entonces reemplaza los valores perdidos por el valor medio de cada columna. Solo se usa con datos numéricos.
* Si el valor es “median”, entonces reemplaza los valores perdidos por el valor de la mediana de cada columna. Solo se usa con datos numéricos.
* Si el valor es “most_frequent”, entonces reemplaza los valores perdidos por el valor más frecuente presente en cada columna. Puede ser usado con datos numéricos y strings.
* Si el valor es “constant”, entonces se reemplaza los valores perdidos por una constante que se especifica en otro parámetro llamado *fill_value*. Puede ser usado con datos numéricos y strings.

## Tratando con datos categóricos
Los datos categóricos (o datos NO numéricos) pueden clasificarse en dos tipos:

* **Ordinales**: Conjuntos de *orden total*. Por ejemplo, las tallas de la ropa es una variable categórica con valores *S,M,L,XL,XXL* y es un conjunto ordenado ya que *S < M < L < Xl < XXl*.
* **Nominales**: No implican un orden. Por ejemplo, los colores.

Bien, existen métodos para trabajar con datos ordinales y datos nominales.

A continuación crearemos un dataset (demo) para ver como tratar con estos tipos de datos.

In [42]:
import pandas as pd
df = pd.DataFrame ([
    ['green', 'M', 10.1, 'class1'],
    ['red', 'L', 13.5, 'class2'],
    ['blue', 'XL', 15.3, 'class1'],
])
df.columns = ['color', 'size', 'price', 'classlabel']

In [43]:
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


### Mapeando características ordinales
Para asegurarnos que nuestros modelos de ML interpreten los datos correctamente, debemos asegurarnos de transformar los datos categóricos a valores enteros.

In [44]:
# Mapear la característica size a un valor entero. Para ello...
# usamos un diccionario que nos facilite hacer el mapeo
size_mapping = {'M' : 1,'L' : 2, 'XL' : 3}
df['size'] = df['size'].map(size_mapping)

In [45]:
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


**Nota**: Si queremos volver a transformar los valores enteros a su correspondiente valor categótico, podemos invertir el proceso de la siguiente manera:

    inv_size_mapping = {v: k for k, v in size_mapping.items()}

y luego la función *map*

### Tratamiento de los labels
Muchas librerías de ML requieren que las clases o etiquetas sean codificadas con valores enteros. Para hacer este mapeo, es posible hacer algo parecido al mapeo de datos ordinales a enteros (sabiendo que la clase o label no es una variable ordinal, por lo que hay que enumerar las clases comenzando con 0)

In [48]:
df2 = df #Requerido para ver dos técnicas

In [71]:
import numpy as np
class_mapping = {label : idx for idx, label in enumerate(np.unique(df['classlabel']))}

In [50]:
class_mapping

{'class1': 0, 'class2': 1}

In [51]:
df['classlabel'] = df['classlabel'].map(class_mapping)

In [52]:
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


**Nota**: Como alternativa a lo anterior (tratamiento de los labels), es posible utilizar *LabelEncoder* de scikit-learn. A continuación el código relacionado.

In [53]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df2['classlabel'].values)

In [54]:
y

array([0, 1, 0], dtype=int64)

In [55]:
df2

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


**Nota1**: El método *fit_transform* es la forma directa de ejecutar *fit* y *transform* por separado.

**Nota2**: Es posible revertir la transformación a partir del arreglo *y* de la siguiente manera:

    class_le.inverse_transform(y)

### One-hot para la codificación de características nominales
Cuando se tienen características nominales el tratamiento es diferente dado que si se aplica lo mismo que se aplicó para las clases (labels), los algoritmos de ML tomarán el conjunto de valores enteros de la transformación como un conjunto de orden total y durante el proceso de entrenamiento el algoritmo asumirá un orden del tipo *a >= b >= c ...*. 

Para evitar esto, se utiliza la tecnica **one-hot**: la idea detrás de este enfoque es crear una nueva función ficticia para cada valor único en la columna de características nominales. Aquí, convertiríamos la característica de color en tres nuevas características: azul, verde y rojo. Los valores binarios se pueden usar para indicar el color particular de una muestra; por ejemplo, una muestra cuyo campo color es blue, se puede codificar como blue = 1, green = 0, red = 0. 

Para realizar esta transformación, podemos usar el *OneHotEncoder* que se implementa en el módulo scikit-learn.preprocessing:

In [65]:
from sklearn.preprocessing import OneHotEncoder
X = df[['color', 'size', 'price']].values
color_le = LabelEncoder()
X[:,0] = color_le.fit_transform(X[:,0])
ohe = OneHotEncoder(categorical_features=[0])
ohe.fit_transform(X).toarray()

C:\Users\dmosquera\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\dmosquera\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


array([[ 0. ,  1. ,  0. ,  1. , 10.1],
       [ 0. ,  0. ,  1. ,  2. , 13.5],
       [ 1. ,  0. ,  0. ,  3. , 15.3]])

Cuando inicializamos OneHotEncoder, definimos la posición de la columna de la variable que queremos transformar a través del parámetro categorical_features (tenga en cuenta que el color es la primera columna en la matriz de características X). De forma predeterminada, OneHotEncoder devuelve una matriz sparse cuando usamos el método transform, por lo que usamos el método toarray() para convertirla en un arreglo Numpy con fines de visualización. Las matrices sparse son simplemente una forma más eficiente de almacenar grandes conjuntos de datos, y son compatibles con muchas funciones de aprendizaje de scikit, lo cual es especialmente útil si contiene muchos ceros. Para omitir el paso de toarray, podríamos inicializar el codificador como OneHotEncoder (..., sparse = False) para devolver una matriz NumPy normal.

In [69]:
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


Una forma aún más conveniente de crear esas características ficticias es con el método *get_dummies* implementado en pandas. Aplicado en un DataFrame, el método get_dummies solo **convertirá columnas de cadena y dejará todas las demás columnas sin cambios**:

In [73]:
import pandas as pd
pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


# Partición del dataset en conjuntos train y test
Para esta parte, utilizaremos el dataset **Wine**, el cual descargamos del repositorio: https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data

Las muestras pertenecen a una de las tres clases diferentes {1, 2, 3}, que corresponden a los tipos diferentes de uvas que se han cultivado en diferentes regiones de Italia.

In [74]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', 
                      header = None)
df_wine.columns = ['Class label', 'Alcohol','Malic acid', 
                   'Ash','Alcalinity of ash', 'Magnesium',
                   'Total phenols', 'Flavanoids','Nonflavanoid phenols',
                   'Proanthocyanins','Color intensity', 
                   'Hue','OD280/OD315 of diluted wines','Proline']

In [75]:
df_wine.head()

,Class label,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [77]:
print('Class labels', np.unique(df_wine['Class label']))

Class labels [1 2 3]


In [78]:
df_wine.shape

(178, 14)

Una forma conveniente de dividir aleatoriamente el dataset en train y test es a través de la función *train_test_split* de scikit-learn:

In [81]:
from sklearn.model_selection import train_test_split
# Separamos los labels
X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

Primero, creamos una matriz NumPy para representar las columnas de características (de la 1 a la 13) en la variable X, además de asignar los labels en la variable y. Luego, usamos la función *train_test_split* para dividir aleatoriamente X e y en conjuntos de datos de prueba y entrenamiento separados. Al establecer *test_size = 0.3*, asignamos el 30% de las muestras de vino a X_test y y_test, y el 70 por ciento restante de las muestras se asigna a X_train y y_train, respectivamente.

# Escalado: colocar todas las características en una misma escala

Hay dos enfoques comunes para poner diferentes características en la misma escala: *normalización* y *estandarización*. La normalización se refiere a la reescala de las características a un rango de [0, 1], que es un caso especial de escalamiento *min-max*. Para normalizar nuestros datos, simplemente podemos aplicar el escalamiento min-max a cada columna.


In [84]:
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
# El ajuste o fit se hace solo para los datos de entrenamiento
X_test_norm = mms.transform(X_test)

Aunque la normalización a través del escalamiento min-max es una técnica comúnmente utilizada, la *estandarización* puede ser más práctica para muchos algoritmos de aprendizaje automático. La razón es que muchos modelos lineales, como la regresión logística y la máquinas de vectores de soporte, inicializan los pesos a cero o a pequeños valores aleatorios cerca de 0. Usando la *estandarización*, en las columnas de características con media 0 y la desviación estándar, se obtienen características que toman la forma de una distribución normal, lo que facilita el aprendizaje de los pesos del modelo. Además, la estandarización mantiene información útil sobre valores atípicos y hace que el algoritmo sea menos sensible a ellos en contraste con el escalamiento min-max, que escala los datos a un rango limitado de valores.

In [85]:
from sklearn.preprocessing import StandardScaler
stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
# El ajuste o fit se hace solo para los datos de entrenamiento
X_test_std = stdsc.transform(X_test)